# SD-JWT for Certification of Registration credential in Proof of Business demo

## Setup cryptographic keys

In [13]:
import asyncio
from ebsi_wallet.did_key import KeyDid, PublicKeyJWK

console = Console()
loop = asyncio.get_event_loop()

# Generate did:key identifier and associated public and privatekey pairs from the cryptographic seed.
cryptographic_seed = b'helloworld'
issuer_did = KeyDid(seed=cryptographic_seed)
issuer_did.create_keypair()

print('Issuer public key JWK:', issuer_did.public_key_jwk)

Issuer public key JWK: {'kty': 'EC', 'kid': 'tJ7_O_Xu862O_9L4oxdBSkRotGJ2Id9smSovHtnUUo8', 'crv': 'P-256', 'x': 'PTKECXSeSzIRRjYBKPECaRBRp1VH_g2EztKLJf8jpVc', 'y': 'HHpPqdZ8P0ZVz9CSryp7FvmtTTnNImmlQuiatJncoNM'}


## Claims to be issued

In [14]:
certificate_of_registration_claims = {
  "legalStatus": "ACTIVE",
  "legalForm": "Aktiebolag",
  "activity": "Real Estate",
  "orgNumber": "559133-2720",
  "registrationDate": "2005-10-08",
  "registeredAddress": {
    "fullAddress": "123 Main St",
    "thoroughFare": "Sveavägen",
    "postName": "Stockholm",
    "locatorDesignator": "111 34",
    "code": "48",
    "adminUnitLevel1": "SE"
  },
  "attestationValidity": "31-Dec-2022"
}

print("Certificate of Registration claims to be issued: ", certificate_of_registration_claims)

Certificate of Registration claims to be issued:  {'legalStatus': 'ACTIVE', 'legalForm': 'Aktiebolag', 'activity': 'Real Estate', 'orgNumber': '559133-2720', 'registrationDate': '2005-10-08', 'registeredAddress': {'fullAddress': '123 Main St', 'thoroughFare': 'Sveavägen', 'postName': 'Stockholm', 'locatorDesignator': '111 34', 'code': '48', 'adminUnitLevel1': 'SE'}, 'attestationValidity': '31-Dec-2022'}


## Calculate full disclosure for claims to be issued

In [15]:
from collections import namedtuple
from ebsi_wallet.util import generate_disclosure_content_and_base64

Disclosure = namedtuple('Disclosure', ['content', 'content_base64'])

disclosures = []
for key, value in certificate_of_registration_claims.items():
    disclosure, disclosure_base64 = generate_disclosure_content_and_base64(key, certificate_of_registration_claims)
    disclosures.append(Disclosure(content=disclosure, content_base64=disclosure_base64))

print("Disclosure content and base64: ", disclosures)

Disclosure content and base64:  [Disclosure(content=['f7d8ca7a66b89771892f6a848e54a0df', 'legalStatus', 'ACTIVE'], content_base64='WyJmN2Q4Y2E3YTY2Yjg5NzcxODkyZjZhODQ4ZTU0YTBkZiIsICJsZWdhbFN0YXR1cyIsICJBQ1RJVkUiXQ'), Disclosure(content=['3831dd79eaf1ed68f5f7070b0ee38a2f', 'legalForm', 'Aktiebolag'], content_base64='WyIzODMxZGQ3OWVhZjFlZDY4ZjVmNzA3MGIwZWUzOGEyZiIsICJsZWdhbEZvcm0iLCAiQWt0aWVib2xhZyJd'), Disclosure(content=['15db0d3d86d0ae0f93477986f4a8b255', 'activity', 'Real Estate'], content_base64='WyIxNWRiMGQzZDg2ZDBhZTBmOTM0Nzc5ODZmNGE4YjI1NSIsICJhY3Rpdml0eSIsICJSZWFsIEVzdGF0ZSJd'), Disclosure(content=['34f3760b814c88db18fe9fd96bb6d734', 'orgNumber', '559133-2720'], content_base64='WyIzNGYzNzYwYjgxNGM4OGRiMThmZTlmZDk2YmI2ZDczNCIsICJvcmdOdW1iZXIiLCAiNTU5MTMzLTI3MjAiXQ'), Disclosure(content=['39313d647ff9ba2cbd59e9445ac173db', 'registrationDate', '2005-10-08'], content_base64='WyIzOTMxM2Q2NDdmZjliYTJjYmQ1OWU5NDQ1YWMxNzNkYiIsICJyZWdpc3RyYXRpb25EYXRlIiwgIjIwMDUtMTAtMDgiXQ'), Disclosure(

## Calculate disclosure SHA-256 digest

In [16]:
import hashlib
import base64

disclosure_sha_256s = []

for disclosure in disclosures:
    sha256_hash = hashlib.sha256()
    sha256_hash.update(disclosure.content_base64.encode('utf-8'))
    sha256_digest = sha256_hash.digest()
    disclosure_sha_256 = base64.urlsafe_b64encode(sha256_digest).decode('utf-8').rstrip("=")
    disclosure_sha_256s.append(disclosure_sha_256)

print("Disclosure SHA-256s: ", disclosure_sha_256s)

Disclosure SHA-256s:  ['86C1ChWguz_vK6cX1-1PeHCXKbwGwmFe30luhEPgoS8', 'WANlmfjQo-VlxAehCFuGyYeJmxJjAgsDOHlbXGdE8UA', 'YnRjoih6JNDc9q-zHu7hg0fCaNmmlqyRvWnzNiV_0RY', 'm5RdTUVAGs6rqlAjm_gNlaAKRJ6-8BhAoyOV-8V6KHA', 'eSCSvAQ3qy-6FuiAuJu2GrZ9hGhlAeWuI9ztD-fQTL4', 'slSGYKk8FzuH9QWh1wFarnA4Ir_RqrMdssMSw_nDPJo', 'TdwGl9TLRDKeWsg_HKF9NNWt8FuwQGyeuizdvINMAuE']


## Create SD-JWT

In [17]:
import json
from jwcrypto import jwt

_sd = disclosure_sha_256s
sd_jwt = issuer_did.generate_sd_jwt(_sd)

print("SD-JWT: ", sd_jwt)

sd_jwt_deserialised = jwt.JWT(key=issuer_did._key, jwt=sd_jwt)
print("SD-JWT payload: ", json.loads(sd_jwt_deserialised.claims))
print("SD-JWT header: ",  json.loads(sd_jwt_deserialised.header))

SD-JWT:  eyJhbGciOiJFUzI1NiJ9.eyJfc2QiOlsiODZDMUNoV2d1el92SzZjWDEtMVBlSENYS2J3R3dtRmUzMGx1aEVQZ29TOCIsIldBTmxtZmpRby1WbHhBZWhDRnVHeVllSm14SmpBZ3NET0hsYlhHZEU4VUEiLCJZblJqb2loNkpORGM5cS16SHU3aGcwZkNhTm1tbHF5UnZXbnpOaVZfMFJZIiwibTVSZFRVVkFHczZycWxBam1fZ05sYUFLUko2LThCaEFveU9WLThWNktIQSIsImVTQ1N2QVEzcXktNkZ1aUF1SnUyR3JaOWhHaGxBZVd1STl6dEQtZlFUTDQiLCJzbFNHWUtrOEZ6dUg5UVdoMXdGYXJuQTRJcl9ScXJNZHNzTVN3X25EUEpvIiwiVGR3R2w5VExSREtlV3NnX0hLRjlOTld0OEZ1d1FHeWV1aXpkdklOTUF1RSJdLCJfc2RfYWxnIjoic2hhLTI1NiIsImV4cCI6MTY4NzI4NjU2OSwiaWF0IjoxNjg3MjgyOTY5LCJpc3MiOiJodHRwczovL2lzc3Vlci5pZ3JhbnQuaW8ifQ.7z0dR5iJbnDaa-G4DW-WKQMweIKS92jyqw5m17BORqHSHWL7JTJs6QvzaLZLPODGiRQlPY2Cp0cRflCy_aHhew
SD-JWT payload:  {'_sd': ['86C1ChWguz_vK6cX1-1PeHCXKbwGwmFe30luhEPgoS8', 'WANlmfjQo-VlxAehCFuGyYeJmxJjAgsDOHlbXGdE8UA', 'YnRjoih6JNDc9q-zHu7hg0fCaNmmlqyRvWnzNiV_0RY', 'm5RdTUVAGs6rqlAjm_gNlaAKRJ6-8BhAoyOV-8V6KHA', 'eSCSvAQ3qy-6FuiAuJu2GrZ9hGhlAeWuI9ztD-fQTL4', 'slSGYKk8FzuH9QWh1wFarnA4Ir_RqrMdssMSw_nDPJo', 'TdwGl9TLRDKeWsg

## Create combined format for issuance (a.k.a issue credential to holder)

*Note: Format is `<SD-JWT>~<Disclosure 1>~<Disclosure 2>~...~<Disclosure N>`*

In [18]:
combined_format_for_issuance = sd_jwt

for disclosure in disclosures:
    combined_format_for_issuance += "~" + disclosure.content_base64

print("Combined format for issuance : ", combined_format_for_issuance)

Combined format for issuance :  eyJhbGciOiJFUzI1NiJ9.eyJfc2QiOlsiODZDMUNoV2d1el92SzZjWDEtMVBlSENYS2J3R3dtRmUzMGx1aEVQZ29TOCIsIldBTmxtZmpRby1WbHhBZWhDRnVHeVllSm14SmpBZ3NET0hsYlhHZEU4VUEiLCJZblJqb2loNkpORGM5cS16SHU3aGcwZkNhTm1tbHF5UnZXbnpOaVZfMFJZIiwibTVSZFRVVkFHczZycWxBam1fZ05sYUFLUko2LThCaEFveU9WLThWNktIQSIsImVTQ1N2QVEzcXktNkZ1aUF1SnUyR3JaOWhHaGxBZVd1STl6dEQtZlFUTDQiLCJzbFNHWUtrOEZ6dUg5UVdoMXdGYXJuQTRJcl9ScXJNZHNzTVN3X25EUEpvIiwiVGR3R2w5VExSREtlV3NnX0hLRjlOTld0OEZ1d1FHeWV1aXpkdklOTUF1RSJdLCJfc2RfYWxnIjoic2hhLTI1NiIsImV4cCI6MTY4NzI4NjU2OSwiaWF0IjoxNjg3MjgyOTY5LCJpc3MiOiJodHRwczovL2lzc3Vlci5pZ3JhbnQuaW8ifQ.7z0dR5iJbnDaa-G4DW-WKQMweIKS92jyqw5m17BORqHSHWL7JTJs6QvzaLZLPODGiRQlPY2Cp0cRflCy_aHhew~WyJmN2Q4Y2E3YTY2Yjg5NzcxODkyZjZhODQ4ZTU0YTBkZiIsICJsZWdhbFN0YXR1cyIsICJBQ1RJVkUiXQ~WyIzODMxZGQ3OWVhZjFlZDY4ZjVmNzA3MGIwZWUzOGEyZiIsICJsZWdhbEZvcm0iLCAiQWt0aWVib2xhZyJd~WyIxNWRiMGQzZDg2ZDBhZTBmOTM0Nzc5ODZmNGE4YjI1NSIsICJhY3Rpdml0eSIsICJSZWFsIEVzdGF0ZSJd~WyIzNGYzNzYwYjgxNGM4OGRiMThmZTlmZDk2YmI2ZDczNCIs